In [52]:
%reload_ext autoreload
%autoreload 2

In [53]:
import os
import pprint

import pandas as pd
import requests_cache
from loguru import logger
from pyprojroot import here
from tqdm.notebook import tqdm

requests_cache.install_cache(str(here()))

os.chdir(here())

In [54]:
from municipal_performance_scraping.transformers import (
    transform_performance_for_evaluations,
)

In [55]:
from municipal_performance_scraping.extractors import extract_performance_data
from municipal_performance_scraping.loaders import load_evaluation
from municipal_performance_scraping.transformers import (
    transform_performance_for_evaluations,
)
from municipal_performance_scraping.transformers import format_json

In [56]:
from municipal_performance_scraping.models import engine

In [50]:
root = pd.json_normalize(format_json(raw_json))
root

,fr,ar,sd,perf,sommedom,annee,maxdom,notedom,id
0,Gouvernance,الحوكمة,"[{'fr': 'Approche Participative', 'ar': 'تطبيق...",21,30,2018,30,21,10
1,Gestion,التصرف,"[{'fr': 'Ressources Humaines', 'ar': 'التصرف ف...",20,30,2018,30,20,11
2,Pérennité,الديمومة,"[{'fr': 'Entretiens des Biens', 'ar': 'تعهد وص...",40,40,2018,40,40,12


In [51]:
from itertools import chain


def unnest(df: pd.DataFrame, level_name: str) -> pd.DataFrame:
    return pd.concat(
        chain.from_iterable(
            [[df], df[level_name].dropna().apply(pd.json_normalize).to_list(),]
        )
    ).drop(columns=level_name)


root.pipe(lambda df: unnest(df, "sd")).pipe(lambda df: unnest(df, "cr"))

,fr,ar,perf,sommedom,annee,maxdom,notedom,id,maxnote,note,crmaxnote,notecrit
0,Gouvernance,الحوكمة,21.0,30.0,2018,30.0,21.0,10,NaN,NaN,NaN,NaN
1,Gestion,التصرف,20.0,30.0,2018,30.0,20.0,11,NaN,NaN,NaN,NaN
2,Pérennité,الديمومة,40.0,40.0,2018,40.0,40.0,12,NaN,NaN,NaN,NaN
0,Approche Participative,تطبيق المقاربة التشاركية,NaN,NaN,NaN,NaN,NaN,1,12,8,NaN,NaN
1,Transparence et acces a l'information,الشفافية والنفاذ إلى المعلومة,NaN,NaN,NaN,NaN,NaN,2,8,8,NaN,NaN
2,Mécanisme de gestion des plaintes,آليات التصرف في الشكاوى,NaN,NaN,NaN,NaN,NaN,3,10,5,NaN,NaN
0,Ressources Humaines,التصرف في الموارد البشرية,NaN,NaN,NaN,NaN,NaN,4,10,10,NaN,NaN
1,Gestion Financière,التصرف في الموراد المالية\r\n,NaN,NaN,NaN,NaN,NaN,5,10,2,NaN,NaN
2,Commandes Publiques\r\n,التصرف في الصفقات العمومية,NaN,NaN,NaN,NaN,NaN,6,10,8,NaN,NaN
0,Entretiens des Biens,تعهد وصيانة ممتلكات الجماعات المحلية,NaN,NaN,NaN,NaN,NaN,7,15,15,NaN,NaN


In [ ]:
dfs = []
for domain in fix(raw_json):
    for sub_domain in domain["sd"]:
        for criterion in sub_domain["cr"]:
            dfs.append(pd.json_normalize(criterion))
        dfs.append(pd.json_normalize(sub_domain))
    dfs.append(pd.json_normalize(domain))

In [ ]:
dfs[-4]

In [ ]:
ids = pd.read_sql("select * from municipalities", con=engine)[
    "municipality_id"
].to_list()

In [ ]:
import concurrent.futures

In [ ]:
transformed = []
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    future_to_mids = {
        executor.submit(extract_performance_data, mid, 2017): mid for mid in ids
    }
    errors = []
    for future in tqdm(concurrent.futures.as_completed(future_to_mids), total=350):
        try:
            mid = future_to_mids[future]
            transformed.append(
                transform_performance_for_evaluations(future.result(), mid, 2017)
            )
        except KeyError:
            errors.append(future_to_mids[future])

In [ ]:
for t in transformed:
    load_evaluation(t)

In [ ]:
transformed = []
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    future_to_mids = {
        executor.submit(extract_performance_data, mid, 2018): mid for mid in ids
    }
    errors = []
    for future in tqdm(concurrent.futures.as_completed(future_to_mids), total=350):
        try:
            mid = future_to_mids[future]
            transformed.append(
                transform_performance_for_evaluations(future.result(), mid, 2018)
            )
        except KeyError:
            errors.append(future_to_mids[future])

In [ ]:
for t in transformed:
    load_evaluation(t)